
Imports


In [2]:
import cv2
import numpy as np
import sys
import math
from moviepy.editor import VideoFileClip, concatenate_videoclips, clips_array

Begin analyzing the video

In [4]:


# Replace 'your_video.mp4' with the name of your MP4 file
input_video_path = 'LVUDMA_benchy_rebinned_36degps_intensity10x.mp4'

# Load the video
cap = cv2.VideoCapture(input_video_path)

# Get the frames per second (FPS)
fps = cap.get(cv2.CAP_PROP_FPS)

# Get the resolution (width and height)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get the total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the duration in seconds
duration = total_frames / fps

degps = 36
sec_per_rot = 360/degps

# Print the extracted information
print(f"Resolution: {width}x{height}")
print(f"FPS: {fps}")
print(f"Duration of loop: {duration} seconds")
print(f'frames per rotation: {fps*sec_per_rot}')

# Release the video capture object
cap.release()


Resolution: 1088x1920
FPS: 18.0
Duration of loop: 240.0 seconds
frames per rotation: 180.0


create a sim based on a video. actual values are bogus

In [5]:
output_video_path = 'output_video.mp4'

# Load the input video
cap = cv2.VideoCapture(input_video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Initialize the starting red frame
red_frame = np.full((height, width, 3), (0, 0, 255), dtype=np.uint8)  # Completely red frame
out_frame = np.full((height, width, 3), (0, 0, 255), dtype=np.uint8)  # Completely red frame

blur_ksize = (25, 25)  # Kernel size for GaussianBlur (width, height) - Adjust as needed
blur_sigma = 0  # Standard deviation in X and Y (0 means it's calculated from kernel size

# Iterate over each frame in the input video
frame_counter = 0
source_weight = 0.0075
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    blurred_frame = cv2.GaussianBlur(frame, blur_ksize, blur_sigma)
    
    
    # Add the negative frame to the red frame (summing pixel values)
    out_frame = cv2.addWeighted(red_frame, 1.0, blurred_frame, source_weight, 0)

    # Write the updated red frame to the output video
    out.write(out_frame)

    # Optional: Print progress
    frame_counter += 1
    source_weight += 0.0015


    print(f'Processed frame {frame_counter}/{total_frames}', end='\r')

# Release resources
cap.release()
out.release()
print("\nVideo processing completed!")

Processed frame 4320/4320
Video processing completed!


combine 2 videos side by side (projector with sim for visualization)

In [6]:
input_video_path1 = 'LVUDMA_benchy_rebinned_36degps_intensity10x.mp4'
input_video_path2 = 'benchy_sim2.mp4'
output_video_path = 'combined_video.mp4'

# Load both input videos
cap1 = cv2.VideoCapture(input_video_path1)
cap2 = cv2.VideoCapture(input_video_path2)


# Get video properties
fps = int(cap1.get(cv2.CAP_PROP_FPS))
width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
width2 = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Ensure both videos have the same height
if height1 != height2:
    # Resize video 2 to match video 1's height
    scale_factor = height1 / height2
    new_width2 = int(width2 * scale_factor)
    cap2.set(cv2.CAP_PROP_FRAME_WIDTH, new_width2)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
combined_width = width1 + width2  # Combined width of both videos
out = cv2.VideoWriter(output_video_path, fourcc, fps, (combined_width, height1))

# Iterate over both videos frame by frame
while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1 or not ret2:
        break

    # Resize the second frame to match the height of the first frame
    #frame2_resized = cv2.resize(frame2, (new_width2, height1))

    # Combine frames side by side
    combined_frame = np.hstack((frame1, frame2))

    # Write the combined frame to the output video
    out.write(combined_frame)

    # Optional: Print progress
    print('Processed frame', end='\r')

# Release resources
cap1.release()
cap2.release()
out.release()
print("\nVideo combination completed!")

Processed frame
Video combination completed!


create the control video


In [7]:
output_video_path = 'control_video.mp4'
target_video_path = 'LVUDMA_benchy_rebinned_36degps_intensity10x.mp4'
meas_video_path = 'benchy_sim2.mp4'
# Load the input video
target_cap = cv2.VideoCapture(target_video_path)
meas_cap = cv2.VideoCapture(meas_video_path)
control_cap = cv2.VideoCapture(target_video_path)

# Get video properties
fps = int(target_cap.get(cv2.CAP_PROP_FPS))
width = int(target_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(target_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(target_cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Initialize the starting red frame


# Iterate over each frame in the input video
frame_counter = 0
control_counter = 180
while target_cap.isOpened():
    ret1, target_frame = target_cap.read()
    ret2, meas_frame = meas_cap.read()

    if not ret1:
        break
    
    
    # Add the negative frame to the red frame (summing pixel values)
    out_frame = cv2.subtract(target_frame,meas_frame)
    gray_frame = cv2.cvtColor(out_frame, cv2.COLOR_BGR2GRAY)
    gray_frame_bgr = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
    

    average_intensity = cv2.mean(gray_frame)[0]

    print(average_intensity)
    if frame_counter == 0:
        target_gray = cv2.cvtColor(target_frame, cv2.COLOR_BGR2GRAY)
        target_intensity = cv2.mean(target_gray)[0]
        first_frame_intensity = average_intensity
        intensity_offset = target_intensity / first_frame_intensity
    if (average_intensity > 0.025*first_frame_intensity):
        ray_frame_bgr = np.clip(gray_frame_bgr * intensity_offset, 0, 255).astype(np.uint8)

        # Write the updated red frame to the output video
        out.write(gray_frame_bgr)

        # Optional: Print progress
        frame_counter += 1
    else:
        break


    print(f'\nProcessed frame {frame_counter}/{total_frames}', end='\r')

# Release resources
target_cap.release()
meas_cap.release()
control_cap.release()
out.release()
print("\nVideo processing completed!")

2.3211310891544117

2.335680912990196/4320

2.322904220281863/4320

2.33513087852328434320

2.35634765625me 4/4320

2.369695925245098/4320

2.385443952971814/4320

2.39538957184436274320

2.4014404296875 8/4320

2.414212814031863/4320

2.4191018497242647/4320

2.4246347464767157/4320

2.4212407130821076/4320

2.4279799517463236/4320

2.4242474724264707/4320

2.4238568474264706/4320

2.4205896714154416/4320

2.4291537415747557/4320

2.4345037722120098/4320

2.4485930798100499/4320

2.4571384803921568/4320

2.4666982613357841/4320

2.4733436734068626/4320

2.4756859872855393/4320

2.4721655752144614/4320

2.4778277228860293/4320

2.4771805108762255/4320

2.4801599839154417/4320

2.4779091030943627/4320

2.4818833295036766/4320

2.4746634689031860/4320

2.4683842677696077/4320

2.4487706801470592/4320

2.4339446423100493/4320

2.4140170228247554/4320

2.4080102060355393/4320

2.3982321346507356/4320

2.3961330039828432/4320

2.3823357077205882/4320

2.3655225566789215/4320

2.341030943627

combine 3 videos 

In [8]:

# Load videos
video1_cap = cv2.VideoCapture('LVUDMA_benchy_rebinned_36degps_intensity10x.mp4')
video2_cap = cv2.VideoCapture('benchy_sim2.mp4')
video3_cap = cv2.VideoCapture('benchy_control.mp4')

# Get video properties (assuming all videos have the same properties)
fps = int(video1_cap.get(cv2.CAP_PROP_FPS))
width = int(video1_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video1_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Prepare VideoWriter for the output video
output = cv2.VideoWriter('combined_video.mp4', fourcc, fps, (width * 3, height))

while True:
    ret1, frame1 = video1_cap.read()
    ret2, frame2 = video2_cap.read()
    ret3, frame3 = video3_cap.read()

    # Check if frames are valid
    if not (ret1 and ret2 and ret3):
        break

    # Resize frames if necessary (make sure they are the same height)
    frame1 = cv2.resize(frame1, (width, height))
    frame2 = cv2.resize(frame2, (width, height))
    frame3 = cv2.resize(frame3, (width, height))

    # Combine frames horizontally
    combined_frame = np.hstack((frame1, frame2, frame3))

    # Write the combined frame to the output video
    output.write(combined_frame)

# Release resources
video1_cap.release()
video2_cap.release()
video3_cap.release()
output.release()
print("Combined video processing completed.")


Combined video processing completed.


In [10]:


# Load the video
video_path = 'LVUDMA_benchy_rebinned_36degps_intensity10x.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Read the first frame
ret, frame = cap.read()

if not ret:
    print("Failed to read video.")
else:
    # Check the number of channels
    if len(frame.shape) == 2:
        print("The video is in grayscale.")
    elif len(frame.shape) == 3 and frame.shape[2] == 3:
        print("The video is in BGR format.")
    else:
        print("Unknown format.")

# Release the video capture object
cap.release()


The video is in BGR format.
